### This is the steps I took to clean the USAir97 data
Library used:  
[Pandas](https://pandas.pydata.org): This is for data manipulations  
[FuzzyWuzzy](https://github.com/seatgeek/fuzzywuzzy):This is for fuzzy string matching  

Dataset referenced:  
[Airports](https://github.com/mwgg/Airports): Github page   
[Open flights](https://openflights.org/data.html): Open source project


In [2]:
import pandas as pd
from fuzzywuzzy import process
import os

In [3]:
with open("./USAir97.net.txt") as f:
    ## You want to replaced the "" from the airport names before spliting
    content = f.read().replace('"', "").split("\n")
## Those are the airports
nodes = content[1:333]

In [4]:
"""
    A little preprocessing.
    Convert Intll, and Intl to international during split.
    You might be able to use regex instead. 
    Other things you could try to replace:
        muni -> municipal
        afb -> Air force base
    You may find more
"""
airports = []
for node in nodes:
    splited = node.split()
    name = splited[1:-3]
    replaceString = ["Intll", "Intl"]
    for st in replaceString:
        if st in name:
            name[name.index(st)] = "International"
    airports.append([splited[0], " ".join(name), splited[-3], splited[-2]])

In [5]:
"""
I used two data base for reference, and loaded them here:
https://github.com/mwgg/Airports
https://openflights.org/data.html

"""

df1 = pd.read_json("./airports.json")
df2 = pd.read_csv("./airports.dat.txt")
df2.columns =  ["Airport ID", "Name","City","Country","IATA","ICAO","Latitude","Longitude","Altitude","Timezone","DST","Tz database time zone","Type","Source"]

usefulColums1 = ["name", "city", "state", "country", "lat", "lon", 'code']
usefulColums2 = ["Name","City", "Country", "Latitude","Longitude","IATA"]

df1 = df1[usefulColums1]
df2=df2[usefulColums2]

df2.columns = ["name", "city", "country", "lat", "lon", 'code']

Compile a list of possible airport names, and created a reverse lookup dictionary, to reduce the runtime

I appened the city name to the end of the airport name to increase our chance of matching

In [7]:
choices1 = []
lookup1 = {}
index = 0
for n, c in df1[["name", "city"]].values:
    choices1.append(str(n)+" "+str(c))
    lookup1[str(n)+" "+str(c)] = index
    index+=1

choices2 = []
lookup2 = {}
index = 0
for n, c in df2[["name", "city"]].values:
    choices2.append(str(n)+" "+str(c))
    lookup2[str(n)+" "+str(c)] = index
    index+=1

For each airport we will try to obtain a match with both database

In [13]:
print("Match result 1:")
print(process.extract("Wiley Post-Will Rogers Mem", choices1, limit=5))
print("Match result 2:")
print(process.extract("Wiley Post-Will Rogers Mem", choices2, limit=5))

Match result 1:
[('Wiley Post Will Rogers Memorial Airport Barrow', 90), ('Will Rogers World Airport Oklahoma City', 86), (' Port Moller', 58), (' Shillong', 56), (' Ipota', 54)]
Match result 2:
[('Wiley Post Will Rogers Memorial Airport Barrow', 90), ('Kingston Norman Rogers Airport Kingston', 86), ('Jackson-Medgar Wiley Evers International Airport Jackson', 86), ('Henry Post Army Air Field (Fort Sill) Fort Sill', 86), ('Will Rogers World Airport Oklahoma City', 86)]


In [261]:
cleaned = []
for airport in airports:
    name = airport[1]
    ## in here we are only extracting the best one, using reverse lookup 
    ## to find out the index of the row for later use.
    match1 = process.extractOne(name, choices1)
    match2 = process.extractOne(name, choices2)
    to_append = [match1[0], lookup1[match1[0]], match1[1], match2[0], lookup2[match2[0]], match2[1]]
    cleaned.append(airport+to_append)
## Move everything into a pandas database
new_df = pd.DataFrame(cleaned, columns=["Id","Name", "x", "y", "m1Name", "m1Idx", "m1Score", "m2Name", "m2Idx", "m2Score"])

In [15]:
new_df.head()

,Id,Name,x,y,m1Name,m1Idx,m1Score,m2Name,m2Idx,m2Score,City,State,lat,lon
0,1,Wiley Post-Will Rogers Mem,0.4407,0.0910,Wiley Post Will Rogers Memorial Airport Barrow,406.0,90,Wiley Post Will Rogers Memorial Airport Barrow,3355.0,90,NaN,NaN,0.0,0
1,2,Deadhorse,0.4830,0.1014,Deadhorse Airport Prudhoe Bay,2659.0,90,Deadhorse Airport Deadhorse,3404.0,90,NaN,NaN,0.0,0
2,3,Ralph Wien Memorial,0.4110,0.1330,Ralph Wien Memorial Airport Kotzebue,2268.0,90,Ralph Wien Memorial Airport Kotzebue,3477.0,90,NaN,NaN,0.0,0
3,4,Fairbanks International,0.4861,0.1528,Fairbanks International Airport Fairbanks,894.0,90,Fairbanks International Airport Fairbanks,3612.0,90,NaN,NaN,0.0,0
4,5,Nome,0.3965,0.1557,Nome Airport Nome,2227.0,90,Nome Airport Nome,3399.0,90,NaN,NaN,0.0,0


### Now we will have manual step
Once the names are matched, I have to mannualy to check the results. The good news is, 88% of the matching are correct, or at least one of the database has a good matching result. I had to hand code about 40 of them. This is where you will discover some of the airport might not exists anymore. Or the airport was matched to another coountry.
Some of the airports might still exists in the database, then you can just findout which row it was in, instead of mannully fill in the information. Hence the last 4 columns in the df above.

In [263]:
last_df = pd.read_excel("./matched_airports.xlsx")

### Now the fun part:
We will compare the score on each matching, and used the one with higher score. 
Please note that chancging the order of the database will make the code below useless. 
Since one database has the states the other one does not, but they both share the IATA code, so you can use the IATA code found from second dataabse to findout the corresponding row in the first database. If it does not exists, then I will use "N/A" as a placeholder

In [288]:
formatted = []
for row in last_df.values:
    basic = []
    for i in row[:4]:
        basic.append(i)
    if not row[-1]==0:
        #Manualy added result
        basic.append(row[1])
        for i in row[-4:]:
            basic.append(i)
    else:
        # Auto matched
        if row[6] >= row[9]:
            cur_row = df1.iloc[int(row[5])]
        else:
            cur_row = df1[df1['code'] == df2[["code"]].iloc[int(row[8])].values[0]]
            if cur_row.size==0:
                cur_row = df2.iloc[int(row[8])]
                cur_row['state'] = "N/A"
                cur_row = cur_row[usefulColums1].values
            else:
                cur_row = cur_row.values[0]
        for val in cur_row[:-1]:
            basic.append(val)
    formatted.append(basic)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/pandas/core/series.py:769: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value


In [289]:
final_result = pd.DataFrame(formatted)

In [290]:
final_result

,0,1,2,3,4,5,6,7,8,9
0,1,Wiley Post-Will Rogers Mem,0.4407,0.0910,Wiley Post Will Rogers Memorial Airport,Barrow,Alaska,United States,71.2892,-156.772000
1,2,Deadhorse,0.4830,0.1014,Deadhorse Airport,Prudhoe Bay,Alaska,United States,70.1975,-148.458000
2,3,Ralph Wien Memorial,0.4110,0.1330,Ralph Wien Memorial Airport,Kotzebue,Alaska,United States,66.8893,-162.610000
3,4,Fairbanks International,0.4861,0.1528,Fairbanks International Airport,Fairbanks,Alaska,United States,64.8183,-147.866000
4,5,Nome,0.3965,0.1557,Nome Airport,Nome,Alaska,United States,64.5125,-165.445000
5,6,St Mary\'s,0.4074,0.1791,Alaska,Saint Marys,Alaska,United States,62.0503,-163.179000
6,7,Aniak,0.4266,0.1837,Aniak Airport,Bethel,Alaska,United States,61.5742,-159.535000
7,8,Anchorage International,0.4752,0.1876,Anchorage International Airport,Anchorage,Alaska,United States,61.1767,-149.961000
8,9,Tuluksak,0.4193,0.1883,Tuluksak,Tuluksak,Alaska,United States,61.0959,-160.968000
9,10,Akiachak,0.4169,0.1901,Akiachak,Akiachak,Alaska,United States,60.9057,-161.424000


## Another Manual step
Now most of our data are cleaned, I had to go through each of them again for sanity check. This is also the time I find out the states for the N/As. 

In [291]:
final_result.to_csv("final_cleaned.csv")

## Done!!
Now we have the final dataset below

In [295]:
new_result = pd.read_csv("final.csv", index_col=0)

In [296]:
new_result

,ID,old_name,x,y,name,city,state,country,lat,lon
0,1,Wiley Post-Will Rogers Mem,0.4407,0.0910,Wiley Post Will Rogers Memorial Airport,Barrow,Alaska,United States,71.28920,-156.772000
1,2,Deadhorse,0.4830,0.1014,Deadhorse Airport,Prudhoe Bay,Alaska,United States,70.19750,-148.458000
2,3,Ralph Wien Memorial,0.4110,0.1330,Ralph Wien Memorial Airport,Kotzebue,Alaska,United States,66.88930,-162.610000
3,4,Fairbanks International,0.4861,0.1528,Fairbanks International Airport,Fairbanks,Alaska,United States,64.81830,-147.866000
4,5,Nome,0.3965,0.1557,Nome Airport,Nome,Alaska,United States,64.51250,-165.445000
5,6,St Mary\'s,0.4074,0.1791,Alaska,Saint Marys,Alaska,United States,62.05030,-163.179000
6,7,Aniak,0.4266,0.1837,Aniak Airport,Bethel,Alaska,United States,61.57420,-159.535000
7,8,Anchorage International,0.4752,0.1876,Anchorage International Airport,Anchorage,Alaska,United States,61.17670,-149.961000
8,9,Tuluksak,0.4193,0.1883,Tuluksak,Tuluksak,Alaska,United States,61.09590,-160.968000
9,10,Akiachak,0.4169,0.1901,Akiachak,Akiachak,Alaska,United States,60.90570,-161.424000
